In [21]:
from tinybig.config import config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'imdb_configs'
config_obj = config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.imdb', 'data_parameters': {'name': 'imdb', 'train_batch_size': 64, 'test_batch_size': 64}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 10, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.0001, 'weight_decay': 1e-05}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.83}}, 'loss_configs': {'loss_class': 'torch.nn.CrossEntropyLoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'device': 'mps', 'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_laye

In [22]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [23]:
print('instantiating objects from config...')
data_obj, model_obj, learner_obj, metric_obj, result_obj = [config_obj.instantiation_from_configs(config['configurations'][f'{stem}_configs'], device=device, class_name=f'{stem}_class', parameter_name=f'{stem}_parameters') for stem in ['data', 'model', 'learner', 'metric', 'output']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  54572


In [24]:
print('loading dataset...')
data_loader = data_obj.load()

loading dataset...


In [25]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...


100%|██████████| 391/391 [00:02<00:00, 153.90it/s, epoch=0/10, loss=0.66, lr=0.0001, metric_score=0.875, time=2.54]  


Epoch: 0, Test Loss: 0.6638417251579597, Test Score: 0.83528, Time Cost: 1.1391239166259766


100%|██████████| 391/391 [00:02<00:00, 169.86it/s, epoch=1/10, loss=0.38, lr=8.3e-5, metric_score=0.9, time=5.98]   


Epoch: 1, Test Loss: 0.40997403162672086, Test Score: 0.85504, Time Cost: 0.9860239028930664


...

Epoch: 7, Test Loss: 0.28815127958727005, Test Score: 0.88476, Time Cost: 0.957024097442627


100%|██████████| 391/391 [00:02<00:00, 172.87it/s, epoch=8/10, loss=0.0873, lr=2.25e-5, metric_score=0.975, time=29.7]


Epoch: 8, Test Loss: 0.2901990436722555, Test Score: 0.88516, Time Cost: 0.9668998718261719


100%|██████████| 391/391 [00:02<00:00, 169.93it/s, epoch=9/10, loss=0.049, lr=1.87e-5, metric_score=0.975, time=33]   


Epoch: 9, Test Loss: 0.2923721988563952, Test Score: 0.88552, Time Cost: 0.9863519668579102
model checkpoint saving to ./ckpt/imdb_configs_checkpoint...


In [26]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [27]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
accuracy 0.88552


In [28]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
